In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

#### **TODO**
* read https://docs.fast.ai/vision.transform.html
* choose between data_block api vs. factory methods
* start training with smaller architectuire (resnet34) lesson 1
* go thru lesson 2 and lesson 3 notebooks(progressive training)
* Read again and look at the code https://github.com/rsomani95/shot-type-classifier/ , https://nbviewer.jupyter.org/github/rsomani95/shot-type-classifier/blob/master/methodology.ipynb
* install collapsible headings, table of contents...https://github.com/minrk/ipython_extensions
* try using VS Code for jupyter nootebooks (remote ssh)